In [ ]:
import pandas as pd
import numpy as np
import os
import time, datetime
import math
import random
from sklearn import preprocessing
import torch
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.classifier import StackingCVClassifier
from scipy.stats import norm, skew
from scipy import stats
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.impute import SimpleImputer
from scipy.misc import derivative
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import linear_model, datasets
from sklearn.linear_model import Lasso
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import lightgbm
from sklearn import svm
from sklearn.neural_network import MLPRegressor,MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
import random
from sklearn.metrics import roc_auc_score
from scipy import optimize
from scipy import special
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn import preprocessing
import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_data_label = pd.read_csv('CCF2021_run_label_Train.csv')
label_p_id = train_data_label['人员编号']
label_c_id = train_data_label['特征码']
answer = {label_p_id[i]:label_c_id[i] for i in range(len(label_p_id))}

In [ ]:
# train_data = pd.read_csv('train_feature.csv')
# test_data = pd.read_csv('test_feature.csv')
train_data = pd.read_csv('/content/drive/MyDrive/ccf data/train_feature.csv')
test_data = pd.read_csv('/content/drive/MyDrive/ccf data/test_feature.csv')

In [ ]:
train_data.shape,test_data.shape

((398338, 21), (196382, 20))

In [ ]:
'''
cols = train_data.columns.tolist()
cols.remove('c_id')
cols.remove('p_id')
cols.remove('label')
for col in cols:
    Min = min(float(train_data[col].min()),float(test_data[col].min()))
    Max = max(float(train_data[col].max()),float(test_data[col].max()))
    train_data.loc[:,col] = (train_data.loc[:,col]-Min)/(Max-Min)
    test_data.loc[:,col] = (test_data.loc[:,col]-Min)/(Max-Min)
'''

"\ncols = train_data.columns.tolist()\ncols.remove('c_id')\ncols.remove('p_id')\ncols.remove('label')\nfor col in cols:\n    Min = min(float(train_data[col].min()),float(test_data[col].min()))\n    Max = max(float(train_data[col].max()),float(test_data[col].max()))\n    train_data.loc[:,col] = (train_data.loc[:,col]-Min)/(Max-Min)\n    test_data.loc[:,col] = (test_data.loc[:,col]-Min)/(Max-Min)\n"

In [ ]:
train_data.head()

,c_id,p_id,pp_number,average_det_time,cid_number,unique_pp_number,unique_average_det_time,unique_lcs,p_number,pid_number,p_speed,c_speed,dis_speed,p_speed_min,c_speed_min,dis_speed_min,Min_time,c_speed_change,p_speed_change,dis_change,label,change_div_p,dis_speed_div_p
0,CY1r8Eov,P0099,0.8,3.437451e+07,558,0,1.000000e+20,2,8,10,8.764114e-10,1.264389e-09,3.879778e-10,0,3.039451e-10,3.039451e-10,1.000000e+30,6,1,-5,0,-0.714286,3.879778e-10
1,CjD8XlTz,P0099,0.8,3.266357e+07,573,0,1.000000e+20,2,8,10,8.764114e-10,1.103917e-09,2.275057e-10,0,3.428395e-10,3.428395e-10,1.000000e+30,6,1,-5,0,-0.714286,2.275057e-10
2,C1b6QpCZ,P0099,0.8,2.081857e+08,538,0,1.000000e+20,2,8,10,8.764114e-10,5.378414e-10,3.385700e-10,0,1.795694e-10,1.795694e-10,1.000000e+30,4,1,-3,0,-0.600000,3.385700e-10
3,C1uhR4YE,P0099,0.8,1.658634e+08,543,0,1.000000e+20,2,8,10,8.764114e-10,1.134907e-09,2.584956e-10,0,1.642113e-10,1.642113e-10,1.000000e+30,3,1,-2,0,-0.500000,2.584956e-10
4,CwhkYNEx,P0099,0.8,2.051248e+08,545,0,1.000000e+20,2,8,10,8.764114e-10,1.652945e-09,7.765340e-10,0,7.457286e-10,7.457286e-10,1.000000e+30,4,1,-3,0,-0.600000,7.765340e-10


In [ ]:
test_data

In [ ]:
def change_div_p(data):
  res = []
  for i,j in zip(data['c_speed_change'],data['dis_change']):
    res.append(j/(i+1))
  return res 
def dis_speed_div_p(data):
  res = []
  for i,j in zip(data['p_speed'],data['dis_speed']):
    res.append(j/(i+1))
  return res 
train_data['change_div_p'] = change_div_p(train_data)
test_data['change_div_p'] = change_div_p(test_data)

train_data['dis_speed_div_p'] = dis_speed_div_p(train_data)
test_data['dis_speed_div_p'] = dis_speed_div_p(test_data)

In [ ]:
p_ids = []
def init(data):
    global p_ids
    p_ids = data['p_id'].unique().tolist()

In [ ]:
init(train_data)

In [ ]:
random.seed(2021)
random.shuffle(p_ids)

In [ ]:
cross_size = 5
n = len(p_ids)//cross_size + 1
p_data = [p_ids[i:i+n] for i in range(0, len(p_ids), n)]
def cross_val(model,train_data):
    global cross_val,p_ids
    acc = 0.0
    for cross in range(cross_size):
        train_p_ids = []
        for i in range(cross_size):
          if i != cross:
            train_p_ids = train_p_ids + p_data[i] 
        test_p_ids = p_data[cross]
        train_index = []
        test_index = []
        # for p_id in p_ids:
        #     if p_id not in train_p_ids:
        #         test_p_ids.add(p_id)
        train_index = train_data[train_data['p_id'].isin(train_p_ids)].index.tolist()
        test_index = train_data[train_data['p_id'].isin(test_p_ids)].index.tolist()
        small_train_data = train_data.iloc[train_index].drop(['c_id','p_id'],axis=1)
        small_test_data = train_data.iloc[test_index]
        test_c_id = small_test_data['c_id'].tolist()
        test_p_id = small_test_data['p_id'].tolist()
        small_test_data.drop(['c_id','p_id'],axis=1,inplace=True)
        model.fit(small_train_data.drop(['label'],axis=1),small_train_data['label'])
        predict_proba_y = model.predict_proba(small_test_data.drop(['label'],axis=1))[:,1:2]
        tmp_answer = {}
        for i in range(len(test_c_id)):
            proba = predict_proba_y[i][0]
            c_id = test_c_id[i]
            p_id = test_p_id[i]
            if proba > tmp_answer.get(p_id,(-1,-1))[1]:
                tmp_answer[p_id] = (c_id,proba)
        true = 0
        for p_id in tmp_answer:
            if tmp_answer[p_id][0] == answer[p_id]:
                true += 1
        print(true/len(test_p_ids))
        acc += true/len(test_p_ids)
    print('acc: ',acc/cross_size)
    # return -acc/cross_size

In [ ]:
def cross_val_two(modela,modelb,train_data):
    global cross_val,p_ids
    acc = 0.0
    for cross in range(cross_size):
        train_p_ids = []
        for i in range(cross_size):
          if i != cross:
            train_p_ids = train_p_ids + p_data[i] 
        test_p_ids = p_data[cross]
        train_index = []
        test_index = []
        # for p_id in p_ids:
        #     if p_id not in train_p_ids:
        #         test_p_ids.add(p_id)
        train_index = train_data[train_data['p_id'].isin(train_p_ids)].index.tolist()
        test_index = train_data[train_data['p_id'].isin(test_p_ids)].index.tolist()
        small_train_data = train_data.iloc[train_index].drop(['c_id','p_id'],axis=1)
        small_test_data = train_data.iloc[test_index]
        test_c_id = small_test_data['c_id'].tolist()
        test_p_id = small_test_data['p_id'].tolist()
        small_test_data.drop(['c_id','p_id'],axis=1,inplace=True)
        modela.fit(small_train_data.drop(['label'],axis=1),small_train_data['label'])
        modelb.fit(small_train_data.drop(['label'],axis=1),small_train_data['label'])
        
        predict_proba_y_a = modela.predict_proba(small_test_data.drop(['label'],axis=1))[:,1:2]
        predict_proba_y_b = modelb.predict_proba(small_test_data.drop(['label'],axis=1))[:,1:2]
        tmp_answer = {}
        for i in range(len(test_c_id)):
            proba = predict_proba_y_a[i][0]+predict_proba_y_b[i][0]
            c_id = test_c_id[i]
            p_id = test_p_id[i]
            if proba > tmp_answer.get(p_id,(-1,-1))[1]:
                tmp_answer[p_id] = (c_id,proba)
        true = 0
        for p_id in tmp_answer:
            if tmp_answer[p_id][0] == answer[p_id]:
                true += 1
        print(true/len(test_p_ids))
        acc += true/len(test_p_ids)
    print('acc: ',acc/cross_size)

In [ ]:
train_data.columns

Index(['c_id', 'p_id', 'pp_number', 'average_det_time', 'cid_number',
       'unique_pp_number', 'unique_average_det_time', 'unique_lcs', 'p_number',
       'pid_number', 'p_speed', 'c_speed', 'dis_speed', 'p_speed_min',
       'c_speed_min', 'dis_speed_min', 'Min_time', 'c_speed_change',
       'p_speed_change', 'dis_change', 'label'],
      dtype='object')

In [ ]:
feature_col = ['c_id', 'p_id', 'pp_number', 'average_det_time', 'cid_number',
       'unique_pp_number', 'unique_average_det_time',  'p_number',
       'pid_number', 'p_speed', 'c_speed', 'dis_speed', 'dis_speed_min', 'Min_time', 'dis_change', 'label']

In [ ]:
%%time
lgb=lightgbm.LGBMClassifier( objective='binary',
                        metric='auc',
                        learning_rate=0.02,
                        n_estimators=500,
                        boosting= "gbdt",
                        max_depth=10,
                        num_leaves=400,
                        feature_fraction = 0.9,  
                        bagging_fraction = 0.95,  
                        bagging_freq = 5, 
                        reg_alpha = 0.02,
                        reg_lambda = 0.05,
                        min_data_in_leaf = 20,
                        )
params =  {'bagging_fraction': 0.9325837890849541, 'bagging_freq': 5, 'boosting_type': 'gbdt', 'feature_fraction': 0.7015355243328362, 'learning_rate': 0.06046174903181386, 'max_depth': 8, 'min_data_in_leaf': 15, 'min_gain_to_split': 0.6093427327673105, 'n_estimators': 500, 'num_leaves': 127, 'reg_alpha': 0.1949793976192661, 'reg_lambda': 0.960591321518608}
params = {'is_unbalance':True,'learning_rate': 0.017737413271505673, 'boosting_type': 'gbdt', 'objective': 'xentropy', 'num_leaves': 63, 'max_depth': 10, 'n_estimators': 500, 'min_data_in_leaf': 34, 'min_gain_to_split': 0.188661079484727, 'reg_alpha': 0.4248068068953985, 'reg_lambda': 1.039347932519718, 'feature_fraction': 0.9715679990466433, 'bagging_fraction': 0.6816784267307361, 'bagging_freq': 5, 'verbose': -1}
# params = {'is_unbalance':True,'learning_rate': 0.017737413271505673, 'boosting_type': 'goss', 'objective': 'xentropy', 'num_leaves': 63, 'max_depth': 10, 'n_estimators': 500, 'min_data_in_leaf': 34, 'min_gain_to_split': 0.188661079484727, 'reg_alpha': 0.4248068068953985, 'reg_lambda': 1.039347932519718}

lgb = lightgbm.LGBMClassifier(**params)
cross_val(lgb,train_data)

0.9175
0.91
0.905
0.91
0.8967254408060453
acc:  0.907845088161209
CPU times: user 2min 28s, sys: 632 ms, total: 2min 28s
Wall time: 2min 28s


In [ ]:
! pip install catboost

In [ ]:
import catboost as cb
catb = cb.CatBoostClassifier(eval_metric="AUC", depth=8, iterations=500, learning_rate=0.02,verbose = 250)
cross_val(catb,train_data)

0:	total: 94.8ms	remaining: 47.3s
250:	total: 24.4s	remaining: 24.2s
499:	total: 47.7s	remaining: 0us
0.93
0:	total: 90.6ms	remaining: 45.2s
250:	total: 24.1s	remaining: 23.9s
499:	total: 47.2s	remaining: 0us
0.9175
0:	total: 91.6ms	remaining: 45.7s
250:	total: 24.3s	remaining: 24.1s
499:	total: 47.5s	remaining: 0us
0.9075
0:	total: 94.3ms	remaining: 47.1s
250:	total: 24.1s	remaining: 23.9s
499:	total: 47.4s	remaining: 0us
0.9125
0:	total: 93.2ms	remaining: 46.5s
250:	total: 24.3s	remaining: 24.1s
499:	total: 47.5s	remaining: 0us
0.9042821158690176
acc:  0.9143564231738035


In [ ]:
cross_val_two(lgb,catb,train_data)

0:	total: 97.7ms	remaining: 48.7s
250:	total: 24.2s	remaining: 24s
499:	total: 47.6s	remaining: 0us
0.925
0:	total: 95.3ms	remaining: 47.6s
250:	total: 24s	remaining: 23.8s
499:	total: 47s	remaining: 0us
0.9075
0:	total: 106ms	remaining: 52.8s
250:	total: 24.3s	remaining: 24.1s
499:	total: 47.3s	remaining: 0us
0.9075
0:	total: 93.2ms	remaining: 46.5s
250:	total: 23.9s	remaining: 23.7s
499:	total: 47s	remaining: 0us
0.9175
0:	total: 92.8ms	remaining: 46.3s
250:	total: 24.3s	remaining: 24.1s
499:	total: 47.4s	remaining: 0us
0.906801007556675
acc:  0.9128602015113352


In [ ]:
fea_ = catb.feature_importances_
fea_name = catb.feature_names_
# plt.figure(figsize=(10, 10))
# plt.barh(fea_name,fea_,height =0.5)
print(pd.DataFrame({
        'column': catb.feature_names_,
        'importance': catb.feature_importances_,
    }).sort_values(by='importance',ascending = False))

                     column  importance
0                 pp_number   20.101257
1          average_det_time   14.376579
14                 Min_time   13.228191
2                cid_number    9.263773
6                  p_number    6.835903
4   unique_average_det_time    6.560508
18             change_div_p    4.923630
15           c_speed_change    3.782508
17               dis_change    3.036780
3          unique_pp_number    2.617010
7                pid_number    2.486301
13            dis_speed_min    2.474012
12              c_speed_min    2.236097
8                   p_speed    2.059522
9                   c_speed    1.880235
10                dis_speed    1.678099
19          dis_speed_div_p    1.465525
16           p_speed_change    0.762401
5                unique_lcs    0.231669
11              p_speed_min    0.000000


In [ ]:
lr = LogisticRegression(# class_weight={0:1,1:100},
                        max_iter=5000,
                        random_state=2018)
cross_val(lr)

0.8275
0.795
0.825
0.8075
0.8060453400503779
acc:  0.8122090680100756


In [ ]:
dtc = DecisionTreeClassifier(max_depth=5,
                            class_weight={0:1,1:1})
abc = AdaBoostClassifier(base_estimator = dtc,
                        n_estimators=500,
                        learning_rate=0.15,
                        random_state=2019)
cross_val(abc)

In [ ]:
# abc.fit(train_data.drop(['c_id','p_id','label'],axis=1),train_data['label'])
model = catb 
model.fit(train_data.drop(['c_id','p_id','label'],axis=1),train_data['label'])

0:	total: 119ms	remaining: 59.4s
250:	total: 29.6s	remaining: 29.3s
499:	total: 57.6s	remaining: 0us


In [ ]:
###false

test_c_id = train_data['c_id']
test_p_id = train_data['p_id']
predict_proba_y = model.predict_proba(train_data.drop(['c_id','p_id','label'],axis=1))[:,1:2]
tmp_answer = {}
train_acc = 0
false_c_id = []
false_p_id = []
for i in range(len(test_c_id)):
    proba = predict_proba_y[i][0]
    c_id = test_c_id[i]
    p_id = test_p_id[i]
    if proba > tmp_answer.get(p_id,(-1,-1))[1]:
        tmp_answer[p_id] = (c_id,proba)
test_p_id = list(set(test_p_id))
for p_id in test_p_id:
    if tmp_answer[p_id][0] == answer[p_id]:
        train_acc += 1
    else:
        false_p_id.append(p_id)
        false_c_id.append(tmp_answer[p_id][0])
print(train_acc,train_acc/len(test_p_id))
false = pd.DataFrame({'false_pids':false_p_id,
                      'false_cids':false_c_id})
# false.to_csv('false.csv',index=False)

1905 0.9539308963445168


In [ ]:
false.sort_values(by='false_pids').head()

,false_pids,false_cids
21,P0006,CzRTjNQP
19,P0007,CLjkwVKy
38,P0017,CsIzxuki
66,P0027,CVUKwvDa
67,P0029,C7b0Jsoz


In [ ]:

test_c_id = test_data['c_id']
test_p_id = test_data['p_id']
predict_proba_y = model.predict_proba(test_data.drop(['c_id','p_id'],axis=1))[:,1:2]
tmp_answer = {}
for i in range(len(test_c_id)):
    proba = predict_proba_y[i][0]
    c_id = test_c_id[i]
    p_id = test_p_id[i]
    if proba > tmp_answer.get(p_id,(-1,-1))[1]:
        tmp_answer[p_id] = (c_id,proba)
test_p_id = list(set(test_p_id))
test_c_id = []
for p_id in test_p_id:
    test_c_id.append(tmp_answer.get(p_id,'-1')[0])
submit = pd.DataFrame({'人员编号':test_p_id,
                       '特征码':test_c_id})
submit.to_csv('output.csv',index=False)

In [ ]:
print(pd.DataFrame({
        'column': test_data.columns[2:], 
        'importance': model.booster_.feature_importance(),
    }).sort_values(by='importance',ascending = False))

AttributeError: ignored

In [ ]:
modela = lgb
modelb = catb 

def get_submit(modela,modelb):
  modela.fit(train_data.drop(['c_id','p_id','label'],axis=1),train_data['label'])
  modelb.fit(train_data.drop(['c_id','p_id','label'],axis=1),train_data['label'])
  test_c_id = test_data['c_id']
  test_p_id = test_data['p_id']
  predict_proba_y_a = modela.predict_proba(test_data.drop(['c_id','p_id'],axis=1))[:,1:2]
  predict_proba_y_b = modelb.predict_proba(test_data.drop(['c_id','p_id'],axis=1))[:,1:2]
  tmp_answer = {}
  for i in range(len(test_c_id)):
      proba = predict_proba_y_a[i][0] +  predict_proba_y_b[i][0]
      c_id = test_c_id[i]
      p_id = test_p_id[i]
      if proba > tmp_answer.get(p_id,(-1,-1))[1]:
          tmp_answer[p_id] = (c_id,proba)
  test_p_id = list(set(test_p_id))
  test_c_id = []
  for p_id in test_p_id:
      test_c_id.append(tmp_answer.get(p_id,'-1')[0])
  submit = pd.DataFrame({'人员编号':test_p_id,
                        '特征码':test_c_id})
  submit.to_csv('output.csv',index=False)
get_submit(modela,modelb)

0:	total: 163ms	remaining: 1m 21s
200:	total: 22.2s	remaining: 33s
400:	total: 43.2s	remaining: 10.7s
499:	total: 53.3s	remaining: 0us


In [ ]:
test_data.columns

Index(['c_id', 'p_id', 'pp_number', 'average_det_time', 'cid_number',
       'unique_pp_number', 'unique_average_det_time', 'unique_lcs'],
      dtype='object')

In [ ]:
import pandas as pd
df_best = pd.read_csv('/content/submit_merge.csv')
df_se = pd.read_csv('submit.csv')
submit = pd.read_csv('/content/output.csv')

In [ ]:
df_best.sort_values('人员编号',inplace=True)
df_se.sort_values('人员编号',inplace=True)
num = 0
for i,j in zip(df_best['特征码'],df_se['特征码']):
    if i!=j:
        num += 1
print(num)

16


In [ ]:
submit 

,人员编号,特征码
592,P0000,CMw8aF7H
919,P0001,C05BKLk4
844,P0002,ChBvxKiO
348,P0003,Cp3g58MG
397,P0004,CXG69dSe
...,...,...
49,P0995,C7SA6ep4
174,P0996,CutQIJ16
278,P0997,CvBP7Mxg
76,P0998,CyUG104t


In [ ]:
# submit = pd.read_csv('/content/output_add_speed.csv')
df_se = pd.read_csv('output.csv')

In [ ]:
num = 0
ansa = {}
ansb = {}
ansc = {}
for pi,pj,i,j in zip(df_best['人员编号'],submit['人员编号'],df_best['特征码'],submit['特征码']):
  ansa[pi] = i 
  ansb[pj] = j
for p,c in zip(df_se['人员编号'],df_se['特征码']):
  ansc[p] = c

p_ans = []
c_ans = []
all_diff = 0
for p in df_best['人员编号']:
  res = [ansa[p],ansb[p],ansc[p]]
  p_ans.append(p)
  rs = ansa[p]
  if ansa[p] != ansb[p] and ansa[p]!=ansc[p] and ansb[p]!=ansc[p]:
    rs = ansa[p] 
    all_diff += 1
  else:
    rs = max(res,key = res.count)
  c_ans.append(rs)
  if rs != ansa[p]:
    num +=1
    print(p,ansa[p],rs)
print(num,all_diff)

0 0


In [ ]:
sbmit = pd.DataFrame({'人员编号':p_ans,
                       '特征码':c_ans})
sbmit.to_csv('submit_13.csv',index=False)

In [ ]:
lightgbm.__version__ 


'2.2.3'